# Symbolic Hermitian Anticommutator

In [1]:
import sympy
from sympy.codegen.ast import Assignment

Define symbols for the minimum number of real variables required to store the 3x3 Hermitian matrices we need to calculate:

$\dfrac{\partial f}{\partial t} = \imath \left[ H, f \right]$

Matrix entries for $f$

In [2]:
fee_R, fuu_R, ftt_R = sympy.symbols('fee_R fuu_R ftt_R', real=True)
feu_R, fet_R, fut_R = sympy.symbols('feu_R fet_R fut_R', real=True)
feu_I, fet_I, fut_I = sympy.symbols('feu_I fet_I fut_I', real=True)

Matrix entries for $\dfrac{\partial f}{\partial t}$

In [3]:
Dfee_R, Dfuu_R, Dftt_R = sympy.symbols('Dfee_R Dfuu_R Dftt_R', real=True)
Dfeu_R, Dfet_R, Dfut_R = sympy.symbols('Dfeu_R Dfet_R Dfut_R', real=True)
Dfeu_I, Dfet_I, Dfut_I = sympy.symbols('Dfeu_I Dfet_I Dfut_I', real=True)

Matrix entries for $H$

In [4]:
Hee_R, Huu_R, Htt_R = sympy.symbols('Hee_R Huu_R Htt_R', real=True)
Heu_R, Het_R, Hut_R = sympy.symbols('Heu_R Het_R Hut_R', real=True)
Heu_I, Het_I, Hut_I = sympy.symbols('Heu_I Het_I Hut_I', real=True)

Define matrices $f$ and $H$ to be Hermitian by construction:

In [5]:
F = sympy.Matrix([[fee_R, feu_R + sympy.I * feu_I, fet_R + sympy.I * fet_I],
                  [feu_R - sympy.I * feu_I, fuu_R, fut_R + sympy.I * fut_I],
                  [fet_R - sympy.I * fet_I, fut_R - sympy.I * fut_I, ftt_R]])

In [6]:
H = sympy.Matrix([[Hee_R, Heu_R + sympy.I * Heu_I, Het_R + sympy.I * Het_I],
                  [Heu_R - sympy.I * Heu_I, Huu_R, Hut_R + sympy.I * Hut_I],
                  [Het_R - sympy.I * Het_I, Hut_R - sympy.I * Hut_I, Htt_R]])

Calculate anticommutator $[H,f] = H \cdot f - f \cdot H$

In [7]:
AC = H*F - F*H

Calculate $\dfrac{\partial f}{\partial t} = \imath \left[ H, f \right]$

In [8]:
dFdt = sympy.I * AC

Assign elements of the $\dfrac{\partial f}{\partial t}$ matrix to the real values we need to store the matrix

In [9]:
iE = 0
iU = 1
iT = 2

dfee_R_dt = Assignment(Dfee_R, sympy.re(dFdt[iE,iE]))
dfuu_R_dt = Assignment(Dfuu_R, sympy.re(dFdt[iU,iU]))
dftt_R_dt = Assignment(Dftt_R, sympy.re(dFdt[iT,iT]))

dfeu_R_dt = Assignment(Dfeu_R, sympy.re(dFdt[iE,iU]))
dfet_R_dt = Assignment(Dfet_R, sympy.re(dFdt[iE,iT]))
dfut_R_dt = Assignment(Dfut_R, sympy.re(dFdt[iU,iT]))

dfeu_I_dt = Assignment(Dfeu_I, sympy.im(dFdt[iE,iU]))
dfet_I_dt = Assignment(Dfet_I, sympy.im(dFdt[iE,iT]))
dfut_I_dt = Assignment(Dfut_I, sympy.im(dFdt[iU,iT]))

Define a function to return a code string for calculating the independent variables we need to store $\dfrac{\partial f}{\partial t}$

In [10]:
def get_rhs_code():
    lines = []

    lines.append(sympy.ccode(dfee_R_dt))
    lines.append(sympy.ccode(dfuu_R_dt))
    lines.append(sympy.ccode(dftt_R_dt))
    
    lines.append(sympy.ccode(dfeu_R_dt))
    lines.append(sympy.ccode(dfet_R_dt))
    lines.append(sympy.ccode(dfut_R_dt))
    
    lines.append(sympy.ccode(dfeu_I_dt))
    lines.append(sympy.ccode(dfet_I_dt))
    lines.append(sympy.ccode(dfut_I_dt))
    
    for l in lines:
        print(l + "\n")

    return "\n".join(lines)

In [11]:
code_string = get_rhs_code()

Dfee_R = -2*Het_I*fet_R + 2*Het_R*fet_I - 2*Heu_I*feu_R + 2*Heu_R*feu_I;

Dfuu_R = 2*Heu_I*feu_R - 2*Heu_R*feu_I - 2*Hut_I*fut_R + 2*Hut_R*fut_I;

Dftt_R = 2*Het_I*fet_R - 2*Het_R*fet_I + 2*Hut_I*fut_R - 2*Hut_R*fut_I;

Dfeu_R = -Hee_R*feu_I - Het_I*fut_R + Het_R*fut_I + Heu_I*fee_R - Heu_I*fuu_R - Hut_I*fet_R + Hut_R*fet_I + Huu_R*feu_I;

Dfet_R = -Hee_R*fet_I + Het_I*fee_R - Het_I*ftt_R - Heu_I*fut_R - Heu_R*fut_I + Htt_R*fet_I + Hut_I*feu_R + Hut_R*feu_I;

Dfut_R = Het_I*feu_R - Het_R*feu_I + Heu_I*fet_R - Heu_R*fet_I + Htt_R*fut_I - Hut_I*ftt_R + Hut_I*fuu_R - Huu_R*fut_I;

Dfeu_I = Hee_R*feu_R + Het_I*fut_I + Het_R*fut_R - Heu_R*fee_R + Heu_R*fuu_R - Hut_I*fet_I - Hut_R*fet_R - Huu_R*feu_R;

Dfet_I = Hee_R*fet_R - Het_R*fee_R + Het_R*ftt_R - Heu_I*fut_I + Heu_R*fut_R - Htt_R*fet_R + Hut_I*feu_I - Hut_R*feu_R;

Dfut_I = -Het_I*feu_I - Het_R*feu_R + Heu_I*fet_I + Heu_R*fet_R - Htt_R*fut_R + Hut_R*ftt_R - Hut_R*fuu_R + Huu_R*fut_R;

